# Improvement: Zero shot data extraction using LLMs, evaluation on training data

In [1]:
import pandas as pd
import json
import ast
from openai import OpenAI
import vllm
import json_metric

In [2]:
model_dir = '/home/bluepurple/.cache/kagglehub/models/metaresearch/llama-3.2/transformers/3b-instruct/1'
llm = vllm.LLM(
    model=model_dir,
    tensor_parallel_size=1, 
    gpu_memory_utilization=0.80, 
    max_model_len=8192,
    trust_remote_code=True,
    dtype="half", 
    enforce_eager=True,
    #distributed_executor_backend="ray",
)
# tokenizer = llm.get_tokenizer()

INFO 08-18 00:06:17 [__init__.py:235] Automatically detected platform cuda.
WARNING 08-18 00:06:22 [config.py:3443] Casting torch.bfloat16 to torch.float16.
INFO 08-18 00:06:22 [config.py:1604] Using max model len 8192
INFO 08-18 00:06:23 [config.py:2434] Chunked prefill is enabled with max_num_batched_tokens=8192.
INFO 08-18 00:06:24 [core.py:572] Waiting for init message from front-end.
INFO 08-18 00:06:24 [core.py:71] Initializing a V1 LLM engine (v0.10.0) with config: model='/home/bluepurple/.cache/kagglehub/models/metaresearch/llama-3.2/transformers/3b-instruct/1', speculative_config=None, tokenizer='/home/bluepurple/.cache/kagglehub/models/metaresearch/llama-3.2/transformers/3b-instruct/1', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config={}, tokenizer_revision=None, trust_remote_code=True, dtype=torch.float16, max_seq_len=8192, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all

Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]


INFO 08-18 00:06:28 [default_loader.py:262] Loading weights took 2.10 seconds
INFO 08-18 00:06:28 [gpu_model_runner.py:1892] Model loading took 6.0160 GiB and 2.454618 seconds
INFO 08-18 00:06:30 [gpu_worker.py:255] Available KV cache memory: 5.55 GiB
INFO 08-18 00:06:30 [kv_cache_utils.py:833] GPU KV cache size: 51,936 tokens
INFO 08-18 00:06:30 [kv_cache_utils.py:837] Maximum concurrency for 8,192 tokens per request: 6.34x
INFO 08-18 00:06:31 [core.py:193] init engine (profile, create kv cache, warmup model) took 2.28 seconds


## prompts

### In system prompt, add further instructions to make model emit cleaner JSON

In [3]:
system_prompt = """As a medical professional, please analyze the following medical notes carefully and extract relevant information in the specified JSON format.
Return ONE valid JSON object only. No code fences. Each key may appear at most once.
Do not include keys that aren’t in the notes. Do not include any values that are not mentioned in the notes."""

In [4]:
few_shot_example ="""
Example:

Clinical Note:\n\nPatient: 29-year-old male\nChief Complaint: Influenza-like symptoms\n\nHistory of Present Illness:\nThe patient presents with multiple symptoms consistent with influenza, but also exhibiting some atypical features. He reports fever, cough, sore throat, chest pain, diarrhea, runny nose, rash, sneezing, and loss of taste and smell. Additionally, the patient mentions frequent urination, night sweats, weight loss, facial pain, and pale skin. The duration and progression of these symptoms are unclear and require further investigation.\n\nPast Medical History:\nNo significant past medical history provided. Further inquiry needed.\n\nVital Signs:\n- Blood Pressure: 105/66 mmHg (slightly low)\n- Temperature: 39.1°C (102.4°F) - Febrile\n- Heart Rate: 101 bpm (Tachycardic)\n- Oxygen Saturation: 99.1% (Within normal range)\n- Glucose Level: 138.2 mg/dL (Slightly elevated)\n\nPhysical Examination:\nGeneral: Patient appears ill, pale skin noted\nHEENT: Facial pain reported, rhinorrhea present\nRespiratory: Cough observed, chest pain reported\nSkin: Rash present, further characterization needed\nAbdomen: Diarrhea reported\n\nAssessment and Plan:\n1. Influenza-like illness: The patient presents with classic flu symptoms including fever, cough, sore throat, and runny nose. However, the presence of additional symptoms warrants further investigation.\n\n2. Loss of taste and smell: This symptom, along with the other respiratory symptoms, raises concern for possible COVID-19 infection. A COVID-19 test should be performed.\n\n3. Rash: The nature and distribution of the rash need to be further evaluated to determine if it's related to the current illness or a separate issue.\n\n4. Frequent urination, night sweats, and weight loss: These symptoms, combined with the slightly elevated glucose level, raise concern for possible diabetes. HbA1c test should be ordered.\n\n5. Facial pain: This could be related to sinusitis, which can occur as a complication of upper respiratory infections. Further examination of sinuses is needed.\n\n6. Chest pain: Given the presence of fever and cough, a chest X-ray should be considered to rule out pneumonia.\n\n7. Diarrhea: While this can occur with influenza, it's not typical. Stool studies may be considered if it persists.\n\n8. Pale skin: In conjunction with other symptoms, this could indicate anemia. A CBC should be ordered.\n\nPlan:\n1. COVID-19 and influenza tests\n2. Chest X-ray\n3. CBC, comprehensive metabolic panel, HbA1c\n4. Symptomatic treatment with antipyretics and hydration\n5. Follow-up in 2-3 days or sooner if symptoms worsen\n6. Isolation precautions until test results are available\n\nThe patient's presentation is complex and may involve multiple systems. Close follow-up and further diagnostic testing are necessary to determine the underlying cause(s) of his symptoms.

JSON:
{
 "patient_info": {
   "age": 29, 
   "gender": "Male"
 }, 
 "visit_motivation": "Influenza (Flu)", 
 "symptoms": ["fever", "cough", "sore_throat", "chest_pain", "diarrhea", "runny_nose", "rash", "sneezing", "loss_of_taste_smell", "frequent_urination", "night_sweats", "weight_loss", "facial_pain", "pale_skin"], 
 "vital_signs": {
   "blood_pressure": {"systolic": {"value": 105, "unit": "mmHg"}, "diastolic": {"value": 66, "unit": "mmHg"}}, 
   "temperature": {"value": 39.1, "unit": "\\u00b0C"}, 
   "heart_rate": {"value": 101, "unit": "bpm"}, 
   "oxygen_saturation": {"value": 99.1, "unit": "%"}, 
   "glucose_level": {"value": 138.2, "unit": "mg/dL"}
   }
}
"""

In [5]:
# Define the template with a placeholder
prompt_template = """
For each entry:
- Accurately identify and extract patient data, visit motivation, symptoms, and vital signs.
- Only include keys that have corresponding information in the notes, omitting any keys that are not mentioned.
- If temperature given in °F, convert to °C: (F-32)*5/9, round to 1 decimal.
- If BP reported as "120/80", map to systolic=120, diastolic=80.
- Follow this JSON structure:

Schema (all keys optional unless mentioned in notes)
{
  "patient_info": {
    "age": number,
    "gender": "male" | "female" | "other" | "unknown"
  },
  "visit_motivation": string,
  "symptoms": [<string>],
  "vital_signs": {
    "heart_rate": {"value":number,"unit": "bpm"},
    "oxygen_saturation": {"value": number, "unit": "%"},
    "cholesterol_level": {"value": number,"unit": "mg/dL"},
    "glucose_level": {"value": number,"unit": "mg/dL"},
    "temperature": {"value": number,"unit": "°C"},
    "respiratory_rate": {"value": number, "unit": "breaths/min"},
    "blood_pressure": {
      "systolic": {"value": number,"unit": "mmHg"},
      "diastolic": {"value": number,"unit": "mmHg"}
    }
  }
}


Here are the clinical notes to review:

{medical_notes}

"""

In [6]:
# load test data
test_df = pd.read_csv("medical-note-extraction-h-2-o-gen-ai-world-ny/test.csv")
train_df = pd.read_csv("medical-note-extraction-h-2-o-gen-ai-world-ny/train.csv")

In [7]:
train_df.head()

,ID,Note,json
0,1,**Clinical Notes**\n\n**Patient Information:**...,"{""patient_info"": {""age"": 41, ""gender"": ""Male""}..."
1,2,Clinical Notes:\n\nPatient: 56-year-old male\n...,"{""patient_info"": {""age"": 56, ""gender"": ""Male""}..."
2,3,Clinical Notes:\n\nPatient: 73-year-old female...,"{""patient_info"": {""age"": 73, ""gender"": ""Female..."
3,4,**Clinical Notes**\n\n**Patient Information:**...,"{""patient_info"": {""age"": 32, ""gender"": ""Female..."
4,5,"**Clinical Notes**\n\n**Patient:** Male, age 5...","{""patient_info"": {""age"": 51, ""gender"": ""Male""}..."


In [8]:
# Try a small subset
train_sample_df = train_df.iloc[:10].copy()

In [9]:
#Build prompts (assemble the system_prompt, prompt template, raw medical notes for every patient)
all_prompts = []
for index,row in train_sample_df.iterrows():
    
    medical_notes = row.Note
    filled_prompt = f"{prompt_template}".replace("{medical_notes}", medical_notes)    
    message=[
        {"role": "system", "content": system_prompt + few_shot_example},
        {"role": "user", "content": filled_prompt}
    ]
    all_prompts.append(message)
    

In [10]:
print(all_prompts[0])

[{'role': 'system', 'content': 'As a medical professional, please analyze the following medical notes carefully and extract relevant information in the specified JSON format.\nReturn ONE valid JSON object only. No code fences. Each key may appear at most once.\nDo not include keys that aren’t in the notes. Do not include any values that are not mentioned in the notes.\nExample:\n\nClinical Note:\n\nPatient: 29-year-old male\nChief Complaint: Influenza-like symptoms\n\nHistory of Present Illness:\nThe patient presents with multiple symptoms consistent with influenza, but also exhibiting some atypical features. He reports fever, cough, sore throat, chest pain, diarrhea, runny nose, rash, sneezing, and loss of taste and smell. Additionally, the patient mentions frequent urination, night sweats, weight loss, facial pain, and pale skin. The duration and progression of these symptoms are unclear and require further investigation.\n\nPast Medical History:\nNo significant past medical history 

In [11]:
def print_outputs(outputs):
    for output in outputs:
        prompt = output.prompt
        generated_text = output.outputs[0].text
        print(f"Generated text: {generated_text!r}")
        print("-" * 80)

## zero shot prediction

#### Use conservative sampling and nuke repetition. 

In [13]:
%%time
#Keep prompts uniform across a batch. vLLM packs better → fewer timeouts/loops.
from vllm import SamplingParams
sampling_params = SamplingParams(n=1,
                                 temperature=0.0,  # maximize determinism for structure
                                 top_p=0.9,
                                 seed=777, # Seed for reprodicibility
                                 max_tokens=1024,      # keep token size tight; oversized budgets induce loops and longer inference time
                                 repetition_penalty=1.15) # reduce "symptom...symptom...symptom" spirals

from time import time
start = time()

outputs = llm.chat(messages=all_prompts,
                   sampling_params=sampling_params,
                   use_tqdm=True)


end = time()
elapsed = (end-start)/60. #minutes
print(f"Inference took {elapsed} minutes!")

Adding requests:   0%|          | 0/10 [00:00<?, ?it/s]

Processed prompts:   0%|                     | 0/10 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.…

Inference took 0.3390666087468465 minutes!
CPU times: user 214 ms, sys: 60.8 ms, total: 275 ms
Wall time: 20.3 s


In [14]:
print_outputs(outputs)

Generated text: '```\n{\n  "patient_info": {\n    "age": 29,\n    "gender": "Male",\n    "visit_motivation": "Unknown",\n    "symptoms": [\n        "anxiety", "atypical_flu", "atypical_flu", "atypical_flu", "abnormal", "hypertension", "flu", "sinus_infection", "tummy", "wheeze", "flu", "flu", "flu", "flu", "flu", "flu", "flu", "flu", "flu", "flu", "flu", "flu", "flu", "flu", "flu", "flu", "flu", "flu", "flu", "flu", "flu", "flu", "flu", "flu", "flu", "flu", "flu", "flu", "flu", "flu", "flu", "flu", "flu", "flu", "flu", "flu", "flu", "flu", "flu", "flu", "flu", "flu", "flu", "flu", "flu", "flu", "flu", "flu", "flu", "flu", "flu", "flu", "flu", "flu", "flu", "flu", "flu", "flu", "flu", "flu", "flu", "flu", "flu", "flu", "flu", "flu", "flu", "flu", "flu", "flu", "flu", "flu", "flu", "flu", "flu", "flu", "flu", "flu", "flu", "flu", "flu", "flu", "flu", "flu", "flu", "flu", "flu", "flu", "flu", "flu", "flu", "flu", "flu", "flu", "flu", "flu", "flu", "flu", "flu", "flu", "flu", "flu", "flu",

## simple cleanup the outputs

------------------------------------ Helper function ------------------------------------------------------

In [15]:
import re
from typing import Any, Optional

In [16]:
def oneside_strip_code_fence(s:str)->str:
    s = re.sub(r"```$","",s.strip())
    s = re.sub(r"^```[a-zA-Z]*\n?", "", s.strip())
    return s

def rm_trailing_commas(s:str)-> str:
    s=  re.sub(r",\s*([\]}])", r"\1", s.strip())
    return s

def balance_brackets(s:str)-> str:
    """Append missing closing braces/brackets if the string is cut off."""
    # cout curly braces
    opens, closes = s.count("{"), s.count("}")
    if closes < opens:
        s+= "}"* (opens-closes)
    opens, closes = s.count("["), s.count("]")
    if closes < opens:
        s+= "]"* (opens-closes)
    return s

def sanitize_text(s: str) -> str:
    s = oneside_strip_code_fence(s)
    s = rm_trailing_commas(s)
    s = balance_brackets(s)
    return s

def json_with_duplicates(s: str):
    """
    Parse JSON but keep duplicated keys as list values.
    """
    def parse_object_pairs(pairs):
        obj = defaultdict(list)
        for k, v in pairs:
            obj[k].append(v)
        # if only one value, unwrap from list
        return {k: v[0] if len(v) == 1 else v for k, v in obj.items()}
    
    return json.loads(s, object_pairs_hook=parse_object_pairs)

In [17]:
def clean_preds(preds:str):
    preds = sanitize_text(preds).split('\n')[1].split('\n')[0]
    return preds

In [18]:
train_sample_df.head()

,ID,Note,json
0,1,**Clinical Notes**\n\n**Patient Information:**...,"{""patient_info"": {""age"": 41, ""gender"": ""Male""}..."
1,2,Clinical Notes:\n\nPatient: 56-year-old male\n...,"{""patient_info"": {""age"": 56, ""gender"": ""Male""}..."
2,3,Clinical Notes:\n\nPatient: 73-year-old female...,"{""patient_info"": {""age"": 73, ""gender"": ""Female..."
3,4,**Clinical Notes**\n\n**Patient Information:**...,"{""patient_info"": {""age"": 32, ""gender"": ""Female..."
4,5,"**Clinical Notes**\n\n**Patient:** Male, age 5...","{""patient_info"": {""age"": 51, ""gender"": ""Male""}..."


In [19]:
json_list = []

for output in outputs:
    res = output.outputs[0].text

    
    try:
        json_list.append(json.loads(sanitize_texts(res)))
    except:
        print("Error")
        print(res)
        print("*"*80)
        json_list.append(res)
    
pred_sample_df = train_sample_df[['ID','json']].copy()
pred_sample_df['json'] = json_list

# test_trial_df.to_csv("test_with_json2_final.csv", index=False)

Error
```
{
  "patient_info": {
    "age": 29,
    "gender": "Male",
    "visit_motivation": "Unknown",
    "symptoms": [
        "anxiety", "atypical_flu", "atypical_flu", "atypical_flu", "abnormal", "hypertension", "flu", "sinus_infection", "tummy", "wheeze", "flu", "flu", "flu", "flu", "flu", "flu", "flu", "flu", "flu", "flu", "flu", "flu", "flu", "flu", "flu", "flu", "flu", "flu", "flu", "flu", "flu", "flu", "flu", "flu", "flu", "flu", "flu", "flu", "flu", "flu", "flu", "flu", "flu", "flu", "flu", "flu", "flu", "flu", "flu", "flu", "flu", "flu", "flu", "flu", "flu", "flu", "flu", "flu", "flu", "flu", "flu", "flu", "flu", "flu", "flu", "flu", "flu", "flu", "flu", "flu", "flu", "flu", "flu", "flu", "flu", "flu", "flu", "flu", "flu", "flu", "flu", "flu", "flu", "flu", "flu", "flu", "flu", "flu", "flu", "flu", "flu", "flu", "flu", "flu", "flu", "flu", "flu", "flu", "flu", "flu", "flu", "flu", "flu", "flu", "flu", "flu", "flu", "flu", "flu", "flu", "flu", "flu", "flu", "flu", "flu", "fl

In [72]:
similarity = json_metric.score(train_sample_df.drop(columns="Note"), pred_sample_df, "ID")
print(similarity)

Error converting to JSON: {'patient_info': {'age': 38, 'gender': 'Male', 'visit_motivation': 'Anemia Evaluation'}, 'visit_motivation': '', 'symptoms': ['Symptom 1: Fever', 'Symptom 1: Fever', 'Symptom 1: Difficulty breathing (dyspnea)', 'Symptom 2: Fatigue', 'Symptom 3: Pale skin', 'Symptom 4: Blurred vision', 'Symptom 5: Wheezing', 'Symptom 6: Dizziness', 'Symptom 7: Vomiting'], 'vital_signs': {'heart_rate': {'value': 114, 'unit': 'bpm'}, 'oxygen_saturation': {'value': 98.4, 'unit': '%'}, 'cholesterol_level': {'value': 132.8, 'unit': 'mg/dL'}, 'glucose_level': {'value': 110.6, 'unit': 'mg/dL'}, 'temperature': {'value': 37.0, 'unit': '°C'}, 'respiratory_rate': {'value': None, 'unit': None}, 'blood_pressure': {'systolic': None, 'diastolic': None}}}
Error converting to JSON: ```
{
  "patient_info": {
    "age": 45,
    "gender": "male",
    "visit_motivation": "",
    "symptoms": [
    "Accur","_ " 
}
```
Error converting to JSON: ```
{
  "patient_info": {
    "age": 53,
    "gender": "f